## Dependencies

In [2]:
import numpy as np
import pandas as pd

from utils import *

## I/O - Initialization

In [3]:
# load the feature dataset as a dataframe
channel_names =  ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "FT7", "FC3", "FCZ", "FC4", "FT8", "T3", "C3", "Cz", "C4", "T4", "TP7", "CP3", "CPz", "CP4", "TP8", "T5", "P3", "PZ", "P4", "T6", "O1", "Oz" , "O2"]

#channel subsets
hop1 = ["Cz","CP3","CPz","P3"]
hop2 = ["Cz","CP3","CPz","P3","FT7","FC3","FCZ","FC4","C4","CP4","TP8","T5","PZ","T6","O1","Oz","O2"]
csv_file = 'eeg_features.csv'
channel_list = hop1
dataset = data_loader(csv_file)
reduced_dataset = channel_selection(dataset, channel_list)
all_features = reduced_dataset.columns[:len(reduced_dataset.columns) - 1]

In [ ]:
#calculate accuracy for each channel
for channel in channel_names:
    print(channel)
    models = ['K-NN', 'GBC']
    dataset = data_loader(csv_file)
    reduced_dataset = channel_selection(dataset, [channel])
    data = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
    for model in models:
        model_training(data, model, stats=False, cm=False, verbose=True)

In [4]:
all_features = reduced_dataset.columns[:len(reduced_dataset.columns) - 1]
#result = incremental_training(dataset=dataset, channel_list=channel_list, feature_subset=all_features, models=['K-NN'], mode='feature', save=True)

## P-Value Thresholding

In [5]:
p_all, p_dict = p_value_thresholding(reduced_dataset, feature_subset=all_features)

/Users/kemalcankucuk/Documents/PAWS Work/eeg-attention/selection.py:77: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat, p_value = stats.ttest_ind(


## Manual Feature Selection

In [6]:
#models = ['K-NN', 'GBC']
models = ['K-NN']
subset_1 = all_features

data = data_preparation(dataset=reduced_dataset, feature_subset=subset_1, pca=True)
for model in models:
    model_training(data, model, stats=False, cm=False, verbose=True)

Accuracy of K-NN classifier on training set: 0.79962894
Accuracy of K-NN classifier on test set: 0.66913580


## Hyperparameter Tuning


### SVM

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix 
 
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, n_jobs= -1) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
print('Result of the best model on the test set: ', grid_predictions)
print(grid.best_params_) 

a = '''
besto_model = SVC(C=10, gamma=0.1, kernel='rbf')
model = besto_model
dataset = data_loader(csv_file)
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)

model.fit(X_train, y_train)
training_acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)
test_acc
'''

### KNN

In [21]:
from sklearn.model_selection import GridSearchCV 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix 
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings("ignore")
 
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
  
# defining parameter range 


param_grid = {'n_neighbors': list(range(89,90)),
              'weights':['uniform','distance'],
              'algorithm':['ball_tree','kd_tree','brute','auto'],
              'p':list(range(1,3)),
              'metric':['cityblock','cosine', 'euclidean', 'haversine', 'l1', 'l2', 'manhattan', 'nan_euclidean']}  

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3, n_jobs= -1) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
print('Result of the best model on the test set: ', grid_predictions)
print(grid.best_params_)




Fitting 5 folds for each of 128 candidates, totalling 640 fits


/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

[CV 4/5] END algorithm=ball_tree, metric=cityblock, n_neighbors=89, p=1, weights=uniform;, score=0.777 total time=   0.6s
[CV 1/5] END algorithm=ball_tree, metric=cityblock, n_neighbors=89, p=1, weights=distance;, score=0.767 total time=   0.6s
[CV 2/5] END algorithm=ball_tree, metric=cityblock, n_neighbors=89, p=1, weights=distance;, score=0.774 total time=   0.7s
[CV 5/5] END algorithm=ball_tree, metric=cityblock, n_neighbors=89, p=1, weights=uniform;, score=0.783 total time=   0.6s
[CV 1/5] END algorithm=ball_tree, metric=cityblock, n_neighbors=89, p=1, weights=uniform;, score=0.758 total time=   0.7s
[CV 2/5] END algorithm=ball_tree, metric=cityblock, n_neighbors=89, p=1, weights=uniform;, score=0.763 total time=   0.7s


/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

[CV 3/5] END algorithm=ball_tree, metric=cityblock, n_neighbors=89, p=1, weights=distance;, score=0.787 total time=   0.7s
[CV 3/5] END algorithm=ball_tree, metric=cityblock, n_neighbors=89, p=1, weights=uniform;, score=0.781 total time=   0.9s


...

[CV 2/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=1, weights=distance;, score=0.774 total time=   0.6s
[CV 1/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=1, weights=distance;, score=0.759 total time=   0.7s
[CV 3/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=1, weights=distance;, score=0.777 total time=   0.7s
[CV 4/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=1, weights=distance;, score=0.792 total time=   0.7s


/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

[CV 5/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=1, weights=distance;, score=0.791 total time=   0.7s
[CV 1/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=uniform;, score=0.751 total time=   0.7s
[CV 2/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=uniform;, score=0.761 total time=   0.7s


/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/homebrew/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


[CV 5/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=uniform;, score=0.779 total time=   0.5s
[CV 3/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=uniform;, score=0.765 total time=   0.7s
[CV 4/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=uniform;, score=0.776 total time=   0.7s
[CV 1/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=distance;, score=0.759 total time=   0.6s
[CV 2/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=distance;, score=0.774 total time=   0.6s
[CV 3/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=distance;, score=0.777 total time=   0.5s
[CV 5/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=distance;, score=0.791 total time=   0.5s
[CV 4/5] END algorithm=auto, metric=nan_euclidean, n_neighbors=89, p=2, weights=distance;, score=0.792 total time=   0.6s
Result of the best model on

In [22]:
print(grid.best_params_)

{'algorithm': 'brute', 'metric': 'cosine', 'n_neighbors': 89, 'p': 1, 'weights': 'distance'}


In [23]:
besto_model = KNeighborsClassifier(algorithm = 'brute',metric='cosine',weights='distance', n_neighbors= 89, p= 1)
model = besto_model
dataset = data_loader(csv_file)
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)

model.fit(X_train, y_train)
training_acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)
test_acc

0.7898640296662547

### DTC

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix 
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings("ignore")
 
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
  
# defining parameter range 
depths = list(range(1, 100))
param_grid = {'max_depth': depths}  

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3, n_jobs= -1) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
test_acc = grid.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)
print(grid.best_params_)

### GBC 

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix 
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings("ignore")
 
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
  
# defining parameter range 
a='''param_grid = {'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [50,65,80,100,115,130,150]}'''

param_grid = {'learning_rate': [1.0],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [50,65,80,100,115,130,150]}
              
grid = GridSearchCV(GradientBoostingClassifier(), param_grid, refit = True, verbose = 3, n_jobs= -1) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
test_acc = grid.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)
print(grid.best_params_)

In [33]:
grid_predictions = grid.predict(X_test)
test_acc = grid.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)
print(grid.best_params_)

Result of the best model on the test set:  0.9237818181818181
{'n_neighbors': 3}


In [ ]:
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)

model = 

In [27]:
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
test_acc = grid.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)
print(grid.best_params_)
model = GradientBoostingClassifier(learning_rate= 0.01,  max_depth= 12, n_estimators= 500)
model.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = model.predict(X_test)
test_acc = model.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)

Result of the best model on the test set:  0.8603213844252163
{'learning_rate': 1.0, 'max_depth': 12, 'n_estimators': 80}
Result of the best model on the test set:  0.8331273176761433


## Yurteri's Method

p_better = []
for i in range(len(accuracies) - 1):
    delta = accuracies[i+1] - accuracies[i]
    if delta <= 0:
        continue
    else:
        
        p_better.append(p_all[i])

data = data_preparation(selected_channels=selected_channels, selected_labels=selected_labels, feature_subset=p_better)
for model in models:
    training, test = model_training(data, model, stats=False, cm=False, verbose=True)

import matplotlib.pyplot as plt
plt.plot(np.linspace(1,len(accuracies),len(accuracies)), accuracies)
plt.legend()
plt.savefig('foo.png', bbox_inches='tight')
plt.grid()
plt.show()
plt.legend(['GBC', 'K-NN', 'SVM', 'DTC', 'NN'])